Installing Necesary Packages

In [1]:
%pip install langchain-chroma
%pip install langchain-openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



Importing Necessary Libraries

In [6]:
import json
import openai
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings

Loading Azure configurable parameters

In [9]:
# load json
f = open("configurable_parameters.json")
params = json.load(f)

# call dict
params_dict = dict(params)

# Configure environment variables  

model_type = params_dict['open_ai_api']['api_type']  
api_key = params_dict['open_ai_api']['api_key'] 
api_endpoint = params_dict['open_ai_api']['api_endpoint']
api_version = params_dict['open_ai_api']['api_version']

Initializing the Embedding Model

In [10]:
embeddings = AzureOpenAIEmbeddings(
    model=model_type,
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=api_endpoint
)

Example Prompt and Example Input-Output Pairs

In [11]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)


examples = [
    {"input": "hot", "output": "cold"},
    {"input": "young", "output": "old"},
    {"input": "strong", "output": "weak"},
    {"input": "light", "output": "dark"},
    {"input": "full", "output": "empty"},
]

Initialize the vector store and embeddings

In [13]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    embeddings,
    Chroma,
    k=1,
)

In [18]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Provide the antonym for the given word:",
    suffix="Input: {input}\nOutput:",
    input_variables=["input"]
)

# Format the prompt with a new input

In [19]:
formatted_prompt = dynamic_prompt.format(input="brave")
print(formatted_prompt)

Provide the antonym for the given word:

Input: strong
Output: weak

Input: brave
Output:
